In [1]:
#I start by importing pandas, numpy modules

In [2]:
import pandas as pd
import numpy as np

In [3]:
# I load the tsv file
df = pd.read_csv('toxicity_annotated_comments_unanimous.tsv', header=0, sep="\t")

In [6]:
# display the first few rows
df.tail()
df.head()

,rev_id,comment,year,logged_in,ns,sample,split
0,527004.0,NEWLINE_TOKENNEWLINE_TOKENNEWLINE_TOKENNEWLINE...,2002,True,user,random,test
1,603474.0,`:Here I cannot answer your questions. I did ...,2003,True,article,random,train
2,808576.0,"NEWLINE_TOKENNEWLINE_TOKENIn fact, I've just d...",2003,True,user,random,dev
3,986363.0,16:13 1 Jun 2003 (UTC),2003,True,article,random,train
4,1433843.0,NEWLINE_TOKENNEWLINE_TOKEN:I replied on .,2003,True,user,random,train
